In [1]:
# pip install langchain langchain-community langchain-openai google-api-python-client google-auth-httplib2 google-auth-oauthlib faiss-cpu pymupdf python-dotenv
from dotenv import load_dotenv; load_dotenv()

False

In [2]:
import io, os, pathlib, mimetypes
from typing import List, Dict, Any, Optional
from pathlib import Path
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate

#### Config

In [ ]:
# Google Drive API 접근 권한 범위(Scope) 지정
# "drive.readonly" : 읽기 전용 접근만 허용
# 다른 권한이 필요하면 Scope를 바꿔야 하지만 RAG 데이터 수집이라면 readonly가 안전하고 충분
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

In [ ]:
# 최초 OAuth 인증 후 발급받은 Access Token / Refresh Token을 로컬에 저장하는 파일
# 한 번 발급받으면 이 토큰을 재사용하기 때문에, 매번 로그인할 필요가 없습니다.
# 토큰이 만료되면 자동으로 Refresh Token으로 재발급합니다.
CREDENTIALS = "credentials.json"

In [ ]:
# Google Drive에서 다운로드한 파일을 저장할 로컬 경로
# 없는 경우 스크립트 생성 시 자동으로 생성
OUTDIR = pathlib.Path("data/raw")

In [ ]:
# 대상 Google Drive 폴더
# 대상 폴더의 ID를 환경변수 DRIVE_FOLDER_ID에서 읽어옵니다.
# 예: https://drive.google.com/drive/folders/<DRIVE_FOLDER_ID>
FOLDER_ID = os.getenv("DRIVE_FOLDER_ID")

In [ ]:
# 공유 드라이브(Shared Drive) 전체의 ID
# 공유 드라이브 내부 폴더를 대상으로 할 때만 필요
# 예: https://drive.google.com/drive/folders/<SHARED_DRIVE_ID>
# 예: 일반 개인 드라이브의 '나와 공유됨' 폴더라면 None으로 둡니다.
SHARED_DRIVE_ID = os.getenv("SHARED_DRIVE_ID")
